In [35]:
using ITensors, ITensorMPS, Optim, LinearAlgebra, Printf, Random, Plots
Random.seed!(123)
include("src/QuantumResource1D.jl")
include("src/HL_module.jl")

N = 5

5

In [36]:
g_values = range(-1.0, 1.0, 25)
all_params, all_costs, all_qr = [], [], []
all_X_exp, all_O_exp, all_ZZ_exp = [], [], []
fidelity_ghz_vals, fidelity_cluster_vals = [], []
λ_vals, delta_E_vals, H_params = [], [], []

for g in g_values
    println("\n=== g = $g ===")

    qr_g = QR1D.QuantumResource1D(N)
    GHZ_wf, params_GHZ, name_GHZ = QR1D.generate_GHZ_state(qr_g)
    cluster_wf, params_cluster, name_cluster = QR1D.generate_cluster_state(qr_g)
    QR1D.AddQR(qr_g, GHZ_wf, params_GHZ, name_GHZ)
    QR1D.AddQR(qr_g, cluster_wf, params_cluster, name_cluster);

    θ_g, cost_g = QR1D.optimize_circuit_QR(qr_g, g)
    psi_g = QR1D.apply_ladder_circuit(qr_g, θ_g)

    hl = HL1D.HamiltonianLearner(psi_g)
    λ_min, v_min = HL1D.LearnHamiltonian_corr_mat(hl)

    push!(all_params, θ_g)
    push!(all_costs, cost_g)
    push!(all_qr, qr_g)
    push!(all_X_exp, real(inner(psi_g', qr_g.aux_ops[1], psi_g)))
    push!(all_O_exp, real(inner(psi_g', qr_g.aux_ops[2], psi_g)))
    push!(all_ZZ_exp, real(inner(psi_g', qr_g.aux_ops[3], psi_g)))
    push!(fidelity_ghz_vals, abs2(inner(psi_g, GHZ_wf)))
    push!(fidelity_cluster_vals, abs2(inner(psi_g, cluster_wf)))

    push!(λ_vals, λ_min)
    push!(H_params, v_min)
    push!(delta_E_vals, hl.E_psi - hl.E_GS)
end



=== g = -1.0 ===
Fidelity wrt to GHZ = 1.0
Fidelity wrt to cluster state = 1.0
Optimizing for g = -1.0...
Iter     Function value   Gradient norm 
     0     1.917353e-32     1.167199e-11
 * time: 5.2928924560546875e-5
Optimizing for g = -1.0...
Iter     Function value   Gradient norm 
     0    -1.000000e+00     4.308552e-09
 * time: 4.887580871582031e-5
Degeneracy = 6
After sweep 1 energy=-2.9999999985326835  maxlinkdim=2 maxerr=3.51E-16 time=0.012
After sweep 2 energy=-2.9999999985326844  maxlinkdim=2 maxerr=3.70E-16 time=0.002
After sweep 3 energy=-2.9999999985327275  maxlinkdim=2 maxerr=3.88E-16 time=0.002
After sweep 4 energy=-2.999999998533207  maxlinkdim=2 maxerr=3.51E-16 time=0.001
After sweep 5 energy=-2.999999998537207  maxlinkdim=2 maxerr=3.70E-16 time=0.002
After sweep 6 energy=-2.999999998563997  maxlinkdim=2 maxerr=5.05E-16 time=0.002
After sweep 7 energy=-2.9999999986965844  maxlinkdim=2 maxerr=4.25E-16 time=0.001
After sweep 8 energy=-2.9999999988086277  maxlinkdim=2 

InterruptException: InterruptException:

In [37]:
# Create plots
p1 = plot(g_values, all_costs, label="Cost", title="Cost vs g", xlabel="g", ylabel="Cost");
p2 = plot(g_values, all_X_exp, label="<X>", title="Expectation values vs g", xlabel="g", ylabel="Expectation",linewidth=1);
plot!(p2, g_values, all_O_exp, label="<O>",linewidth=1);
plot!(p2, g_values, all_ZZ_exp / (N - 1), label="<ZZ>",linestyle=:dash,linewidth=3);

p3 = plot(g_values, fidelity_ghz_vals, label="GHZ fidelity", title="Fidelity vs g", xlabel="g", ylabel="Fidelity");
plot!(p3, g_values, fidelity_cluster_vals, label="cluster state fidelity");

display(p1)
display(p2)
display(p3)

@show fidelity_ghz_vals
@show fidelity_cluster_vals

fidelity_ghz_vals = Any[0.062499999836197986, 0.0722053251653323, 0.08313737826764266, 0.0953770232014215, 0.10904074141920864, 0.12430376639185618, 0.14142371247126725, 0.16077106385568482, 0.182881707274608, 0.22118091012867086, 0.2642977567155897, 0.30977151065009817, 0.35075901477129295, 0.34230296997092086, 0.369471003986688]
fidelity_cluster_vals = Any[0.9999999999999997, 0.9975248548299688, 0.989828061190224, 0.9766927480565426, 0.9581379754831754, 0.9343583297086531, 0.905645917683413, 0.8723228385344544, 0.8346917215510683, 0.5988269623802157, 0.426827363790124, 0.32712207202220833, 0.2564256092610961, 0.010240479720369688, 0.009786358489342131]


15-element Vector{Any}:
 0.9999999999999997
 0.9975248548299688
 0.989828061190224
 0.9766927480565426
 0.9581379754831754
 0.9343583297086531
 0.905645917683413
 0.8723228385344544
 0.8346917215510683
 0.5988269623802157
 0.426827363790124
 0.32712207202220833
 0.2564256092610961
 0.010240479720369688
 0.009786358489342131

In [38]:

@show HL_fidelities
@show g_values[1]

# Create plots
pl1 = plot(g_values, delta_E_vals, label="delta E", title="Energy relative to GS g", xlabel="g", ylabel="delta E")
pl2 = plot(g_values, λ_vals, label="λ_vals", xlabel="g", ylabel="λ")


display(pl1)
display(pl2)


HL_fidelities = Any[0.4792105485521182, 1.0000000000000004, 0.9999999999999996, 0.9999999999999991, 0.9999999999999991]
g_values[1] = -1.0


attempt to save state beyond implementation limit


In [40]:
# Extract 14th element from each v_min vector
param4_vals = [v[14] for v in H_params]
param14_vals = [v[4] for v in H_params]

pl1 = plot(g_values, param14_vals, label="ZXZ", xlabel="g", ylabel="strength")
plot!(pl1, g_values, param4_vals, label="ZZ", xlabel="g", ylabel="strength")




attempt to save state beyond implementation limit
attempt to save state beyond implementation limit
attempt to save state beyond implementation limit
